# openai api key 테스트

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에 저장된 환경변수 로드
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 라이브러리 로딩 및 객체 생성

In [ ]:

from openai import OpenAI

# openai api 인증 및 openai 객체 생성
client = OpenAI(api_key=OPENAI_API_KEY)

## TTS(text to speech)

In [ ]:
with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="alloy",
    input = "오늘 아침 날씨는 1도 따뜻하게 입어야해"
) as response:
    response.stream_to_file("speech_alloy.wav")

In [ ]:
# 다른 음성으로 생성

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="cedar",
    input = "오늘 아침 날씨는 1도 따뜻하게 입어야해"
) as response:
    response.stream_to_file("speech_cedar.wav")

## STT(Speech to Text) 

In [ ]:
# 소리 파일

audio_file = open("speech_alloy.wav", "rb")
type(audio_file)

In [ ]:
# STT(Speech to Text)

response = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    # response_format="json",
    # response_format="verbose_json",
    response_format="text", 
)

In [ ]:
response

In [ ]:
# dict로 변환, json의 경우에 가능
# dict(response)

In [ ]:
# text 포맷 응답 결과
# '오늘 아침 날씨는 1도, 따뜻하게 입어야 해.\n'
response.replace('\n','')   # \n 제거

- STT 수행 시, 파일 클 경우 (25MB 이상)

## Windows에서 FFmpeg 설치
1. Chocolatey 설치
Chocolatey가 설치되어 있지 않다면 다음 단계를 따라 설치할 수 있습니다:

    1) 관리자 권한으로 PowerShell을 엽니다.

    2) 다음 명령어를 실행하여 Chocolatey를 설치합니다:
        
URL에서 복사해서 설치
      
https://chocolatey.org/install  

  3) Chocolatey 설치가 완료되면, 다음 명령어로 FFmpeg를 설치할 수 있습니다:

choco install ffmpeg


2. vs-code, 터미널 모두 닫았다가 다시 시작

In [ ]:
from pydub import AudioSegment
import math
import openai

# 오디오 파일 불러오기
audo_file = "datas/audio.mp3"
song = AudioSegment.from_mp3(audo_file)

# Whisper는 약 25MB 제한이 있으므로, 시간 기준으로 나누기
# 파일 크기에 따라 다르지만, 보통 10분(600초) 정도면 20MB 내외
chunk_length_ms = 10 * 60 * 1000  # 10분 단위로 자르기 (단위: 밀리초)
num_chunks = math.ceil(len(song) / chunk_length_ms)

print(f"총 {num_chunks}개의 조각으로 분할합니다.")

# 전체 텍스트를 담을 리스트
full_transcript = []

for i in range(num_chunks):
    start = i * chunk_length_ms
    end = min((i + 1) * chunk_length_ms, len(song))
    chunk = song[start:end]
    chunk_filename = f"chunk_{i}.mp3"
    chunk.export(chunk_filename, format="mp3")
    
    print(f"{chunk_filename} 변환 중...")

    # Whisper API 호출
    with open(chunk_filename, "rb") as audio_file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="text",
        )
        full_transcript.append(response)
        print(f"{chunk_filename} 변환 완료.")

# 모든 조각의 텍스트를 합치기
final_text = "\n".join(full_transcript)

# 결과 저장
with open("full_transcript.txt", "w", encoding="utf-8") as f:
    f.write(final_text)

print("전체 파일의 텍스트 변환이 완료되었습니다!")